In [1]:
project_name="mice"

In [2]:
import os

In [3]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\ML_Ops_learning\\Mice-Protein-Expression'

## config/config.yaml

In [6]:
%%writefile config/config.yaml
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://archive.ics.uci.edu/static/public/342/mice+protein+expression.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

Overwriting config/config.yaml


# Entity Folder
1. Goto entity folder  src-> wine_quality -> entity-> __init__.py 
2. paste below code in __init__.py constructor file

In [7]:
project_name

'mice'

In [9]:
#%%writefile -a src/{project_name}/entity/__init__.py
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Config folder

1. Copy entire ConfigurationManager class
2. Goto config folder src->wine_quality->config->configuration.py and paste entire ConfigurationManager class in configuration.py
3. Import DataIngestionConfig from wine_quality.entity using this line  
from wine_quality.entity import DataIngestionConfig

In [10]:
project_name

'mice'

In [15]:
#%%writefile -a src/{project_name}/config/__init__.py
from mice.constants import *
from mice.utils import read_yaml, create_directories

Appending to src/mice/config/__init__.py


In [16]:
# to remove the warning in the  noteook
from mice.constants import PARAMS_FILE_PATH,CONFIG_FILE_PATH

In [18]:
# %%writefile -a src/{project_name}/config/__init__.py
# from mice.entity import DataIngestionConfig

In [20]:
#%%writefile -a src/{project_name}/config/__init__.py


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# Component folder

1. Copy entire DataIngestion class
2. Goto Component folder [ src-> wine_quality -> components - > data_ingestion.py ]
2. Paste DataIngestionclass in data_ingestion.py.py
3. Import dataIngestionconfig from entity using this line 

from projectname.entity import dataIngestionconfig

In [21]:
project_name

'mice'

In [23]:
#%%writefile -a src/{project_name}/components/data_ingestion.py
import os
import urllib.request as request
import zipfile
from pathlib import Path
from mice import logger
from mice.utils import get_size

In [25]:
# %%writefile -a src/{project_name}/components/data_ingestion.py
# from mice.entity import DataIngestionconfig

In [28]:
#%%writefile -a src/{project_name}/components/data_ingestion.py

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-30 12:00:12,434] - mice - INFO - __init__ file - LineNum:32   - yaml file: config\config.yaml loaded successfully
[2023-11-30 12:00:12,448] - mice - INFO - __init__ file - LineNum:32   - yaml file: params.yaml loaded successfully
[2023-11-30 12:00:12,450] - mice - INFO - __init__ file - LineNum:52   - created directory at: artifacts
[2023-11-30 12:00:12,454] - mice - INFO - __init__ file - LineNum:52   - created directory at: artifacts/data_ingestion


[2023-11-30 12:00:19,669] - mice - INFO - 4047717794 file - LineNum:15   - artifacts/data_ingestion/data.zip download! with following info: 
Server: nginx/1.25.3
Date: Thu, 30 Nov 2023 06:30:14 GMT
Transfer-Encoding: chunked
Connection: close




## Into file
pipeline/stage_01_data_ingestion.py

In [30]:
project_name

'mice'

In [31]:
%%writefile src/{project_name}/pipeline/stage_01_data_ingestion.py
from mice import logger
from mice.config import ConfigurationManager
from mice.components.data_ingestion import DataIngestion

class DataIngestionTrainingPipeline:
    def __init__(self):
        pass

    def main(self):
        config = ConfigurationManager()
        data_ingestion_config = config.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()

Overwriting src/mice/pipeline/stage_01_data_ingestion.py


## Into file
main

In [39]:
%%writefile -a main.py
from mice import logger
from mice.pipeline.stage_01_data_ingestion import DataIngestionTrainingPipeline


STAGE_NAME = "Data Ingestion stage"
try:
   logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<") 
   data_ingestion = DataIngestionTrainingPipeline()
   data_ingestion.main()
   logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
        logger.exception(e)
        raise e

Overwriting main.py


In [38]:
os.system("python main.py")

0